<a href="https://colab.research.google.com/github/AlexMontgomerie/deepLearning/blob/master/denoise_additive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
#!rm -rf deepLearning
!git clone https://github.com/AlexMontgomerie/deepLearning
%cd deepLearning
!git pull origin master
!pip install gputil
!pip install psutil
!pip install humanize
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

/content
fatal: destination path 'deepLearning' already exists and is not an empty directory.
/content/deepLearning
From https://github.com/AlexMontgomerie/deepLearning
 * branch            master     -> FETCH_HEAD
Already up to date.
  Using cached https://files.pythonhosted.org/packages/ed/0e/5c61eedde9f6c87713e89d794f01e378cfd9565847d4576fa627d758c554/GPUtil-1.4.0.tar.gz
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
('RAM Free: 10.7 GB', ' | Proc size: 150.4 MB')
GPU RAM Free: 4015MB | Used: 7426MB | Util  65% | Total 11441MB


In [2]:
from common import *
!./setup.sh

Using TensorFlow backend.


In [0]:

import sys
import json
import os
import glob
import keras
from keras import regularizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Input, UpSampling2D, concatenate, Subtract
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet
import matplotlib.pyplot as plt

def get_denoise_model_additive(shape):

  inputs = Input(shape)
  
  # convolution layers
  conv3  = Conv2D(8, 3 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv5  = Conv2D(8, 5 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv7  = Conv2D(8, 7 , activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  conv11 = Conv2D(8, 11, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  
  # network
  net = Subtract()([inputs, conv3])
  net = BatchNormalization()(net)
  net = Subtract()([inputs, conv5])
  net = BatchNormalization()(net)
  net = Subtract()([inputs, conv7])
  net = BatchNormalization()(net)
  net = Subtract()([inputs, conv11])
  net = BatchNormalization()(net)  

  return Model(inputs = inputs, outputs = net)